# Single Flight Simulation - Complete Workflow

This notebook demonstrates a detailed single flight simulation workflow.

## Topics Covered

- Configuration management
- Component building and validation
- Flight simulation execution
- Comprehensive result analysis
- Data export in multiple formats
- Visualization suite

In [24]:
from pathlib import Path
import matplotlib.pyplot as plt

from src.config_loader import ConfigLoader
from src.motor_builder import MotorBuilder
from src.environment_setup import EnvironmentBuilder
from src.rocket_builder import RocketBuilder
from src.flight_simulator import FlightSimulator
from src.visualizer import Visualizer
from src.data_handler import DataHandler
from src.validators import RocketValidator, MotorValidator, EnvironmentValidator

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ All imports successful")

✓ All imports successful


## Part 1: Load and Validate Configuration

In [25]:
config_path = Path('../configs/single_sim/01_minimal.yaml')

config_loader = ConfigLoader()
config_loader.load_from_yaml(str(config_path))

rocket_cfg = config_loader.get_rocket_config()
motor_cfg = config_loader.get_motor_config()
env_cfg = config_loader.get_environment_config()
sim_cfg = config_loader.get_simulation_config()

print(f"Configuration loaded: {rocket_cfg.name}")

Configuration loaded: Simple Rocket


In [26]:
config_path = Path('../configs/single_sim/02_complete.yaml')

config_loader = ConfigLoader()
config_loader.load_from_yaml(str(config_path))

rocket_cfg = config_loader.get_rocket_config()
motor_cfg = config_loader.get_motor_config()
env_cfg = config_loader.get_environment_config()
sim_cfg = config_loader.get_simulation_config()

print(f"Configuration loaded: {rocket_cfg.name}")

Configuration loaded: Calisto


## Part 2: Build Motor

In [27]:
motor_builder = MotorBuilder(motor_cfg)
motor = motor_builder.build()

print(motor_builder.get_summary())


{'total_impulse_ns': 5466.977135499999, 'average_thrust_n': 2189.4181559871845, 'burn_time_s': 2.497, 'propellant_mass_kg': 2.955911961392022, 'total_mass_kg': 4.770911961392022, 'max_thrust_n': 2663.168, 'max_thrust_time_s': 0.877}


## Part 3: Build Environment

In [28]:
env_builder = EnvironmentBuilder(env_cfg)
environment = env_builder.build()

print(env_builder.get_summary())

{'location': '(39.3897°, -8.2889°)', 'elevation_m': 100.0, 'atmospheric_model': 'standard_atmosphere', 'wind_velocity_ms': 5.0, 'wind_direction_deg': 90.0, 'gravity_ms2': 9.80665, 'sea_level_pressure_pa': 101325.0, 'sea_level_temperature_k': 288.15, 'sea_level_density_kg_m3': 1.225000018124288, 'elevation_pressure_pa': 100150.95938216125, 'elevation_temperature_k': 287.50112102350226, 'elevation_density_kg_m3': 1.213538832684671}


## Part 4: Build Rocket

In [29]:
rocket_builder = RocketBuilder(rocket_cfg, motor=motor, motor_config=motor_cfg)
rocket = rocket_builder.build()

print(rocket_builder.get_summary())

stability = rocket_builder.get_stability_info()
print(f"\nStability Margin: {stability['static_margin_calibers']:.2f} calibers")
print(f"Status: {'✓ STABLE' if stability['static_margin_calibers'] > 1 else '✗ UNSTABLE'}")

{'name': 'Calisto', 'radius_m': 0.0635, 'length_m': 2.031, 'dry_mass_kg': 17.241, 'total_mass_kg': 20.196911961392022, 'static_margin_calibers': 8.698611673981073, 'has_motor': True, 'has_nose_cone': True, 'has_fins': True, 'fin_count': 4, 'has_parachute': True, 'has_air_brakes': False}

Stability Margin: 8.70 calibers
Status: ✓ STABLE


## Part 5: Run Flight Simulation

In [31]:
simulator = FlightSimulator(
    rocket=rocket,
    config=sim_cfg,
    environment=environment
    #rail_length_m=sim_cfg.rail_length_m,
    #inclination_deg=sim_cfg.inclination_deg,
    #heading_deg=sim_cfg.heading_deg
)

flight = simulator.run()
simulator.print_summary()


FLIGHT SIMULATION SUMMARY

--- ALTITUDE METRICS ---
  Apogee:                2538.63 m
  Apogee Time:           21.74 s
  Apogee Location:       (-3.2, 407.4) m

--- VELOCITY METRICS ---
  Max Velocity:          255.44 m/s
  Max Mach Number:       0.755
  Off-Rail Velocity:     35.25 m/s
  Impact Velocity:       -5.28 m/s

--- IMPACT METRICS ---
  Impact Location:       (-3.3, 440.4) m
  Lateral Distance:      440.4 m
  Flight Duration:       456.75 s

--- STABILITY ---
  Initial Margin:        8.70 calibers
  Off-Rail Margin:       9.16 calibers



## Part 6: Generate All Visualizations

In [41]:
output_dir = Path('notebooks_outputs/single_flight')
visualizer = Visualizer(output_dir)


visualizer.create_standard_plots(trajectory_data=simulator.get_trajectory_data())
print(f"All plots saved to {output_dir}")

All plots saved to notebooks_outputs/single_flight


## Part 7: Export Data

In [ ]:
data_handler = DataHandler()

# Export everything
summary = simulator.get_summary()

data_handler.export_complete_dataset(
    flight=flight,
    summary=summary,
    output_dir=str(output_dir),
    formats=['csv', 'json']
)

print(f"Data exported to {output_dir}")